In [43]:
import numpy as np
import random
from IPython.core.debugger import set_trace

class Convolution2D:
                     
    def __init__(self, n_in, n_out, kernel, padding="valid", stride=(1,1), W=None, b=None):
        self.n_in = n_in
        self.n_out = n_out
        assert padding == "valid" or padding == "same"
        self.padding = padding 
        self.pads = None
        self.stride = stride
        
        assert kernel != None and len(kernel) == 2
        self.kernel = kernel

        if W is None:
            k_h, k_w = kernel
            size = n_out * n_in * k_h * k_w 
            weights = self._init_weights(size)
            weights = np.array(weights).reshape(n_out, n_in, k_h, k_w)
            scale = np.sqrt(2./size)
            W = weights * scale       
        if b is None:
            b = np.zeros(n_out)
       
        self.W = W
        self.b = b
        self.X = None
        self.Y = None
        self.X_shape = None
        self.Y_shape = None
        
        self.g_W = None
        self.g_b = None
        self.g_X = None
        self.g_Y = None
        
        return
   
    def _init_weights(self, num):
        w=[]
        for i in range(num): 
            w.append(random.uniform(-1, 1))
        
        return w   

    def _compute_shape_and_pads(self):
        X_shape = self.X.shape
        x_h = X_shape[1]
        x_w = X_shape[2]
        
        s_h = self.stride[0]
        s_w = self.stride[1]
        k_h = self.kernel[0]
        k_w = self.kernel[1]
        
        if self.padding == "valid":
            y_h = int(np.ceil((x_h - k_h + 1) / s_h))
            y_w = int(np.ceil((x_w - k_w + 1) / s_w))
            zeros_h, zeros_w = (0,0)
        else: # "same"
            y_h = int(np.ceil(x_h / s_h))
            y_w = int(np.ceil(x_w / s_w))
            
            # (y-1): index of last number of y, (y-1)*s: mapped to index in x
            # (y-1)*s+k: count of needed elements of x, like (y-1)*s+1 + k-1 
            # (y-1)*s+k-x: count of extra elements from x, possibly negative when x is enough to cover
            zeros_h = max((y_h - 1) * s_h + k_h - x_h, 0)
            zeros_w = max((y_w - 1) * s_w + k_w - x_w, 0)

        pad0 = zeros_h // 2
        pad1 = zeros_h - pad0
        pad2 = zeros_w // 2
        pad3 = zeros_w - pad2
        self.pads = (pad0, pad1, pad2, pad3)
        
        self.Y_shape = (self.n_out, y_h, y_w)
        self.X_shape = (self.n_in, x_h+zeros_h, x_w+zeros_w) 
        
        return
    
    def _pad(self, X):

        if (np.array(self.pads) == 0).all():
            return X

        h_top = self.pads[0]
        h_bot = self.pads[1]
        w_left = self.pads[2]
        w_rigt = self.pads[3]
                
        x_h = X.shape[1]
        x_w = X.shape[2]
        new_h = x_h + h_top + h_bot
        new_w = x_w + w_left + w_rigt

        padded_X = np.zeros(shape=(n_in, new_h, new_w))
        padded_X[:, h_top:-h_bot, w_left:-w_rigt] = X
        
        return padded_X

    def _convolve2D(self, x, k):
        y_h = self.Y_shape[1]
        y_w = self.Y_shape[2]
        y = np.empty(shape=(y_h, y_w))
        k_h = k.shape[0]
        k_w = k.shape[1]
        s_h = self.stride[0]
        s_w = self.stride[1]

        for i in range(y_h):
            for j in range(y_w):
                r0 = i * s_h
                r1 = r0 + k_h
                c0 = j * s_w
                c1 = c0 + k_w
                y[i,j] = (x[r0:r1, c0:c1] * k).sum()
        
        return y
    
    def forward(self, X):
    
        assert X.ndim == 3
        assert (np.array(X[0].shape) > np.array(self.kernel)).all()

        self.X = X
        if self.X_shape is None:
            self._compute_shape_and_pads() 
        else:
            assert X.shape == self.X_shape

        self.X = self._pad(X)
        Y = np.zeros(shape=self.Y_shape)
        
        for i in range(self.n_out):
            for j in range(self.n_in):
                k = self.W[i,j]
                x = self.X[j]
                Y[i] += self._convolve2D(x, k)
             
            Y[i] += self.b[i]
        
        self.Y = Y
        return Y
    
    def backward(self, g_Y):
        
        self.g_Y = g_Y
        g_W = np.zeros(shape=(n_out, n_in, kernel[0], kernel[1]))
        g_b = np.zeros(n_out)
        
        y_h = self.Y_shape[1]
        y_w = self.Y_shape[2]

        s_h = self.stride[0]
        s_w = self.stride[1]
        k_h = self.W.shape[2]
        k_w = self.W.shape[3]

        for i_Y in range(self.n_out):
            g_y = self.g_Y[i_Y]
            for i_X in range(self.n_in):
                x = self.X[i_X]
                for i_kh in range(k_h):
                    for i_kw in range(k_w):
                        hs = i_kh; he = hs + y_h * s_h
                        ws = i_kw; we = ws + y_w * s_w
                        g_W[i_Y,i_X,i_kh,i_kw] = (g_y * x[hs:he:s_h, ws:we:s_w]).sum()
        
            g_b[i_Y] = g_y.sum()

        g_X = np.zeros(shape=self.X_shape)

        for i_yh in range(y_h):
            i_xh = i_yh * s_h
            for i_yw in range(y_w):
                i_xw = i_yw * s_w
                for i_Y in range(self.n_out):
                    g_X[:, i_xh:i_xh+k_h, i_xw:i_xw+k_w] += self.W[i_Y] * g_Y[i_Y, i_yh, i_yw]  
            
        self.g_Y = g_Y
        self.g_W = g_W
        self.g_b = g_b
        self.g_X = g_X
        
        return g_X
    
    def __str__(self):

        
        s = "\nX is:" + ("" if self.X is None else str(self.X.shape))
        s += "\n" + str(self.X)
        s += "\npadding is: " + str(self.pads)
        s += "\nY is:" + ("" if self.Y is None else str(self.Y.shape))
        s += "\n" + str(self.Y)
        s += "\nkernel is: " + str(self.W.shape)
        s += "\n" + str(self.W)
        s += "\nbias is: " + str(self.b.shape)
        s += "\n" + str(self.b)
        s += "\nstride is:\n" + str(self.stride)
        
        s += "\ng_Y is:" + ("" if self.g_Y is None else str(self.g_Y.shape))
        s += "\n" + str(self.g_Y)
        s += "\ng_W is:" + ("" if self.g_W is None else str(self.g_W.shape))
        s += "\n" + str(self.g_W)
        s += "\ng_b is:" + ("" if self.g_b is None else str(self.g_b.shape))
        s += "\n" + str(self.g_b)
        s += "\ng_X is:" + ("" if self.g_X is None else str(self.g_X.shape))
        s += "\n" + str(self.g_X)
        return s

# when we set all the kernels, biases, g_Y as ones, the resulted g_X shows 
# how many times one input x element has been used in forwarding

In [46]:
n_out = 2; n_in = 2; kernel = (2,3); stride = (1,3); padding="same"
W = np.ones(shape=(n_out, n_in, kernel[0], kernel[1]))
bias = np.ones(n_out)

c = Convolution2D(n_out, n_in, kernel=kernel, padding=padding, stride=stride, W=W, b=bias)
print(c)

a = np.ones(15).reshape(3,5)
b = np.array([a,a])
print(b)

c.forward(b)
print(c)

g = np.ones(shape=c.Y_shape)
print(g)
c.backward(g)
print(c)


X is:
None
padding is: None
Y is:
None
kernel is: (2, 2, 2, 3)
[[[[ 1.  1.  1.]
   [ 1.  1.  1.]]

  [[ 1.  1.  1.]
   [ 1.  1.  1.]]]


 [[[ 1.  1.  1.]
   [ 1.  1.  1.]]

  [[ 1.  1.  1.]
   [ 1.  1.  1.]]]]
bias is: (2,)
[ 1.  1.]
stride is:
(1, 3)
g_Y is:
None
g_W is:
None
g_b is:
None
g_X is:
None
[[[ 1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.]]

 [[ 1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.]
  [ 1.  1.  1.  1.  1.]]]

X is:(2, 4, 6)
[[[ 1.  1.  1.  1.  1.  0.]
  [ 1.  1.  1.  1.  1.  0.]
  [ 1.  1.  1.  1.  1.  0.]
  [ 0.  0.  0.  0.  0.  0.]]

 [[ 1.  1.  1.  1.  1.  0.]
  [ 1.  1.  1.  1.  1.  0.]
  [ 1.  1.  1.  1.  1.  0.]
  [ 0.  0.  0.  0.  0.  0.]]]
padding is: (0, 1, 0, 1)
Y is:(2, 3, 2)
[[[ 13.   9.]
  [ 13.   9.]
  [  7.   5.]]

 [[ 13.   9.]
  [ 13.   9.]
  [  7.   5.]]]
kernel is: (2, 2, 2, 3)
[[[[ 1.  1.  1.]
   [ 1.  1.  1.]]

  [[ 1.  1.  1.]
   [ 1.  1.  1.]]]


 [[[ 1.  1.  1.]
   [ 1.  1.  1.]]

  [[ 1.  1.  1.]
   [ 1.  1.  1.]]]]
bias